<a href="https://colab.research.google.com/github/Meghana1303/Artificial-Neural-Networks/blob/master/cats_dogs(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '/content/drive/My Drive/Colab Notebooks/cats-and-dogs/train'
validation_data_dir = '/content/drive/My Drive/Colab Notebooks/cats-and-dogs/test'
nb_train_samples = 40
nb_validation_samples = 10
epochs = 10
batch_size = 8



In [0]:
if K.image_data_format() == 'channels_first':
  input_shape = (3, img_width, img_height)
else:
  input_shape = (img_width, img_height, 3)
  


In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [0]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [0]:
model.compile(loss='binary_crossentropy', # or categorical_crossentropy
              optimizer='rmsprop',# or adagrad
               metrics=['accuracy'])

In [0]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

print(train_generator.class_indices)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save('model.h5')

Found 40 images belonging to 2 classes.
{'cat': 0, 'dog': 1}
Found 10 images belonging to 2 classes.


Epoch 1/10
5/5 [==============================] - 17s 3s/step - loss: 0.7716 - acc: 0.4500 - val_loss: 0.7460 - val_acc: 0.5000
Epoch 2/10
5/5 [==============================] - 1s 244ms/step - loss: 0.8491 - acc: 0.5750 - val_loss: 0.6783 - val_acc: 0.5000
Epoch 3/10
5/5 [==============================] - 2s 307ms/step - loss: 0.8077 - acc: 0.3750 - val_loss: 0.6869 - val_acc: 0.5000
Epoch 4/10
5/5 [==============================] - 1s 280ms/step - loss: 0.7212 - acc: 0.5000 - val_loss: 0.6921 - val_acc: 0.5000
Epoch 5/10
5/5 [==============================] - 2s 320ms/step - loss: 0.7220 - acc: 0.4250 - val_loss: 0.6945 - val_acc: 0.3750
Epoch 6/10
5/5 [==============================] - 1s 281ms/step - loss: 0.6709 - acc: 0.5750 - val_loss: 2.0395 - val_acc: 0.0000e+00
Epoch 7/10
5/5 [==============================] - 2s 310ms/step - loss: 0.6856 - acc: 0.5750 - val_loss: 0.6880 - v

In [0]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
from os import listdir
from os.path import isfile, join


# dimensions of our images
img_width, img_height = 150, 150

In [0]:
# load the model we saved
model = load_model('model.h5')
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])   

mypath = "/content/drive/My Drive/Colab Notebooks/cats-and-dogs/test/cat/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)
# predicting images
dog_counter = 0 
cat_counter  = 0
for file in onlyfiles:
    img = image.load_img(mypath+file, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict_classes(images, batch_size=10)
    classes = classes[0][0]
    
    if classes == 0:
        print(file + ": " + 'cat')
        cat_counter += 1
    else:
        print(file + ": " + 'dog')
        dog_counter += 1
print("Total Dogs :",dog_counter)
print("Total Cats :",cat_counter)

['11.jpg', '34.jpg', '36.jpg', '38.jpg', '45.jpg']
11.jpg: cat
34.jpg: cat
36.jpg: dog
38.jpg: dog
45.jpg: dog
Total Dogs : 3
Total Cats : 2


In [0]:
#Confution Matrix and Classification Report
from sklearn.metrics import classification_report
Y_pred = model.predict_generator(validation_generator, nb_validation_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['Cats', 'Dogs']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

Confusion Matrix
[[5 0]
 [5 0]]
Classification Report
              precision    recall  f1-score   support

        Cats       0.50      1.00      0.67         5
        Dogs       0.00      0.00      0.00         5

    accuracy                           0.50        10
   macro avg       0.25      0.50      0.33        10
weighted avg       0.25      0.50      0.33        10



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
